# Ce projet fonctionne avec les versions des package suivants:

- matplotlib==3.9.0
- pandas==2.2.2
- seaborn==0.13.2
- missingno==0.5.2
- scikit-learn==1.5.1

## The Data
This is a dataset of food products listing the ingredients and nutritional facts of more than 300,000 foods from over 150 countries in the world. The data is meant to be free to the public to help users decode food labels, and make better food choices in general. The dataset contains more than 300,000 rows over 163 columns. However, as we will see there are a lot of missing or obviously incorrect values.<br/>
Generalities:

- fields that end with _t are dates in the UNIX timestamp format (number of seconds since Jan 1st 1970)
- fields that end with _datetime are dates in the iso8601 format: yyyy-mm-ddThh:mn:ssZ
- fields that end with _tags are comma separated list of tags (e.g. categories_tags is the set of normalized tags computer from the categories field)
- fields that end with a language 2 letter code (e.g. fr for French) is the set of tags in that language
- fields that end with _100g correspond to the amount of a nutriment (in g, or kJ for energy) for 100 g or 100 ml of product
- fields that end with _serving correspond to the amount of a nutriment (in g, or kJ for energy) for 1 serving of the product

Depuis le 26 juin 2024, la méthode de calcul du Nutri-Score a été mise à jour on peu en trouver les détails dans ce document:
- https://www.santepubliquefrance.fr/content/download/150262/file/FAQ-updatedAlgo-FR_20231222.pdf

En utilisant ce document, on peut selectionner les colonnes à conserver les plus pertinente.

In [1209]:
import numpy as np
import pandas as pd
from scipy import stats


import statsmodels.api as sm
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.ensemble import IsolationForest
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

import tkinter as tk
from tkinter import filedialog

import missingno as msno
import ipywidgets as widgets
from wordcloud import WordCloud
from IPython.display import display, HTML,clear_output

# Nettoyer et Filtrer les Features et Produits

## 1.1. Collecter et Préparer les Données Brutes
### 1. Charger les données dans un DataFrame Pandas

In [1379]:
def project_init():
    global original_data, original_selected
    display(HTML("<h1 style='color: navy'>Open Food Facts</h1>"))
    print('Initialisation du projet...', end='\r')
    filepath ='datas/openfoodfacts.csv'
    target ='nutrition_grade_fr'
    original_selected=['energy_100g','saturated-fat_100g','sugars_100g','salt_100g','proteins_100g','fiber_100g']
    # selected = ['energy_100g','salt_100g','alcohol_100g','cocoa_100g','potassium_100g','nutrition_grade_fr']
    original_data = pd.read_csv(f"{filepath}", sep="\t", encoding="utf-8", on_bad_lines="skip", low_memory=False)
    print(' '*len('Initialisation du projet...'), end='\r')
    print('Projet chargé')
    return filepath, target, selected, original_data

In [1381]:
filepath, target, selected, original_data= project_init()

Projet chargé              


In [1382]:
def select_file():
    print("charger le nouveau fichier")
    root =tk.Tk()
    root.withdraw()

    filepath = filedialog.askopenfilename(
        title="Choisissez votre fichier csv",
        filetypes=(("Fichiers CSV","*.csv"), ("Tous les fichiers","*.*"))
    )
    print(' '*len("charger le nouveau fichier"),end='\r')
    print("chargement en cours...")
    root.destroy()

    if filepath:
        try:
            original_data = pd.read_csv(f"{filepath}", sep="\t", encoding="utf-8", on_bad_lines="skip", low_memory=False)
            print(' '*len("chargement en cours..."),end='\r')
            print(f"Fichier chargé avec succès depuis le chemin: {filepath}")
            return original_data
        except Exeption as e:
            print("Erreur lors du chargement du fichier:",e)
    else:
        print("Aucun fichier sélectionné.")

In [1383]:
def split_features(data):
    quantitative_features = data.select_dtypes(include=['float64','int64'])
    qualitative_features = data.select_dtypes(include=['object'])
    return quantitative_features,qualitative_features

In [1384]:
def columns_to_keep(data, selected, target):
    columns_list=['code','product_name']
    selected = selected + columns_list
    # print(target)
    # print(selected)
    if target not in selected:
        selected.append(target)

    filtered_data = data[selected]
    print(filtered_data.columns)
    return filtered_data

In [1389]:
def clean_row_without_target(filtered_data,target):
    # print(f'targte==> {target}')
    # print(f'filtered_data==> {filtered_data.columns}')
    data.shape
    print(f"Nombre de lignes avant suppression : {data.shape[0]}")

    display(HTML("<h4>suppression des lignes sans cible...</h4>"))

    data_cleaned = filtered_data.dropna(subset=[target])
    # print(' '*len('suppression des lignes sans cible...'), end='\r')
    print('✅ Lignes sans cible supprimées')
    print(f"Nombre de lignes après suppression des ligne sans la cible '{target}' : {data_cleaned.shape[0]}")
    return data_cleaned

In [1391]:
def nan_detection(data):
    display(HTML("<h4>Nombre de valeur NaN des features selectionnées:</h4>"))

    nan_counts= data[original_selected].isna().sum()
    print(nan_counts)
    return nan_counts

In [1409]:
def nan_inputation(data):
    exeption_col = ['energy_100g']
    # Sélectionner les colonnes se terminant par '_100g'
    columns_to_modify = [col for col in data.filter(regex='_100g').columns if col not in exeption_col]
    
    # Créer une copie des données
    nan_inputation = data.copy()
    
    # Remplacer les valeurs supérieures à 100 ou inférieures à 0 par NaN uniquement dans les colonnes sélectionnées
    nan_inputation.loc[:, columns_to_modify] = nan_inputation.loc[:, columns_to_modify].apply(
        lambda x: np.where((x > 100) | (x < 0), np.nan, x)
    )
    
    return nan_inputation

In [1411]:
def my_app():
    data = original_data.copy()
    selected = original_selected
    # filepath, target, selected, data= project_init()
    shape = data.shape
    print(f'Shape :\nLe jeu de données contient {shape[0]} lignes et {shape[1]} colonnes\n')
    print(f'Info :\nLes information sur les colonnes du jeu de données sont les suivantes:\n')
    data.info()
    print(f'\nDescribe :\nInformation sur les différentes colonnes du jeu de données :\n')
    # print(data.describe())
    
    quantitative_features,qualitative_features = split_features(data)
    
    filtered_data = columns_to_keep(data,selected,target)
    
    # print(selected)
    data_cleaned=clean_row_without_target(filtered_data,target)
    nan_detection(data_cleaned)
    
    data_cleaned=nan_inputation(data_cleaned)
    
    nan_detection(data_cleaned)
    data_cleaned.head()
    return data_cleaned

In [1413]:
my_app()

Shape :
Le jeu de données contient 320772 lignes et 162 colonnes

Info :
Les information sur les colonnes du jeu de données sont les suivantes:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320772 entries, 0 to 320771
Columns: 162 entries, code to water-hardness_100g
dtypes: float64(106), object(56)
memory usage: 396.5+ MB

Describe :
Information sur les différentes colonnes du jeu de données :

Index(['energy_100g', 'saturated-fat_100g', 'sugars_100g', 'salt_100g',
       'proteins_100g', 'fiber_100g', 'code', 'product_name',
       'nutrition_grade_fr'],
      dtype='object')
Nombre de lignes avant suppression : 320772


✅ Lignes sans cible supprimées
Nombre de lignes après suppression des ligne sans la cible 'nutrition_grade_fr' : 221210


energy_100g             191
saturated-fat_100g      191
sugars_100g             191
salt_100g               191
proteins_100g           191
fiber_100g            27440
dtype: int64


energy_100g             191
saturated-fat_100g      193
sugars_100g             205
salt_100g               243
proteins_100g           193
fiber_100g            27442
dtype: int64


,energy_100g,saturated-fat_100g,sugars_100g,salt_100g,proteins_100g,fiber_100g,code,product_name,nutrition_grade_fr
1,2243.0,28.57,14.29,0.00000,3.57,3.6,0000000004530,Banana Chips Sweetened (Whole),d
2,1941.0,0.00,17.86,0.63500,17.86,7.1,0000000004559,Peanuts,b
3,2540.0,5.36,3.57,1.22428,17.86,7.1,0000000016087,Organic Salted Nut Mix,d
7,1833.0,4.69,15.62,0.13970,14.06,9.4,0000000016124,Organic Muesli,c
12,2230.0,5.00,3.33,1.60782,16.67,6.7,0000000016872,Zen Party Mix,d
...,...,...,...,...,...,...,...,...,...
320742,1477.0,0.00,2.35,0.03048,1.18,4.7,9787461062105,Natural Cassava,a
320751,1643.0,0.60,2.60,0.68000,13.00,5.9,9847548283004,Tartines craquantes bio au sarrasin,a
320757,2406.0,3.73,3.89,0.10000,21.22,12.2,9900000000233,Amandes,b
320763,21.0,0.20,0.50,0.02540,0.50,0.2,99111250,Thé vert Earl grey,c
